## 네이버 무비 리뷰 분류 모형(임베딩 적용) 

- 약 2만건의 네이버 무비 리뷰 데이터를 활용해 Sentiment Classification을 하는 모형을 만들어 본다. 
- 임베딩과 다른 기법을 이용해 성능 향상 여부를 확인한다. 

In [23]:
import pandas as pd
import numpy as np
from konlpy.tag import Mecab
from mxnet.gluon import nn, rnn
from mxnet import gluon, autograd
import gluonnlp as nlp
from mxnet import nd 
import mxnet as mx
import multiprocessing as mp
import time
import itertools
from tqdm import tqdm


mecab = Mecab()


### Vocab 생성 

학습셋 전체의 문장을 이용해 전처리를 한 뒤, Vocab을 생성한다. `Mecab` 형태소 분석기로 형태소만으로 Vocab을 생성 

In [24]:
rating = pd.read_csv("ratings.txt",sep='\t')

In [25]:
rating.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [26]:
dataset = [(d, l) for d,l in zip(rating['document'], rating['label'])]

In [27]:
seq_len = 30

In [28]:
length_clip = nlp.data.PadSequence(seq_len, pad_val="<pad>")

def preprocess(data):
    comment, label = data
    morphs = mecab.morphs(str(comment).strip())
    return(length_clip(morphs), label)

def preprocess_dataset(dataset):
    start = time.time()
    with mp.Pool() as pool:
        dataset = gluon.data.SimpleDataset(pool.map(preprocess, dataset))
    end = time.time()
    print('Done! Tokenizing Time={:.2f}s, #Sentences={}'
          .format(end - start, len(dataset)))
    return dataset

In [29]:
preprocessed  = preprocess_dataset(dataset)

Done! Tokenizing Time=9.73s, #Sentences=200000


첫번째 문장의 첫 11개 토큰 출력  

In [30]:
preprocessed[0][0][:11]

['어릴', '때', '보', '고', '지금', '다시', '봐도', '재밌', '어요', 'ㅋㅋ', '<pad>']

학습셋 전체로 토큰 빈도를 생성 `counter`를 만들고, `vocab`을 생성. 
문장 생성이나 seq2seq가 아니기 때문에 `bos_token`, `eos_token` 표현은 생략 

In [31]:
counter = nlp.data.count_tokens(itertools.chain.from_iterable([c for c, _ in preprocessed]))

vocab = nlp.Vocab(counter, bos_token=None, eos_token=None, min_freq=15)

In [32]:
vocab.idx_to_token[:10]

['<unk>', '<pad>', '.', '이', '는', '영화', '다', '고', '하', '의']

In [33]:
fasttext_simple = nlp.embedding.create('fasttext', source='wiki.ko')

In [34]:
vocab.set_embedding(fasttext_simple)

### 학습셋 생성 

토큰을 `index`로 변환 하여 학습을 위한 데이터로 변환 

In [35]:
preprocessed_encoded  = [(vocab[data], label)  for data, label in preprocessed ]

In [36]:
train, test = nlp.data.train_valid_split(preprocessed_encoded, valid_ratio=0.1)

In [53]:
batchify_fn = nlp.data.batchify.Tuple(nlp.data.batchify.Stack(),
                                      nlp.data.batchify.Stack('float32'))

train_dataloader  = gluon.data.DataLoader(train, batch_size=100, batchify_fn=batchify_fn, shuffle=True, last_batch='discard')
test_dataloader  = gluon.data.DataLoader(test, batch_size=100, batchify_fn=batchify_fn, shuffle=True, last_batch='discard')

### 모델 정의 

In [93]:
class SentClassificationModelAtt(gluon.HybridBlock):
    def __init__(self, vocab_size, num_embed, hidden_size, **kwargs):
        super(SentClassificationModelAtt, self).__init__(**kwargs)
        self.hidden_size = hidden_size 
        with self.name_scope():
            self.embed = nn.Embedding(input_dim=vocab_size, output_dim=num_embed)
            self.drop = nn.Dropout(0.3)
            self.lstm = rnn.LSTM(self.hidden_size, dropout=0.2, bidirectional=True, layout="NTC")
            self.attention = nlp.model.MLPAttentionCell(30, dropout=0.2)
            self.dense = nn.Dense(2)  
    def hybrid_forward(self, F ,inputs):
        em_out = self.drop(self.embed(inputs))
        bilstmout = self.lstm(em_out)
        #Self Attention : http://freesearch.pe.kr/archives/4876 
        ctx_vector, _ = self.attention(bilstmout, bilstmout)
        outs = self.dense(ctx_vector) 
        return(outs)

In [94]:
ctx = mx.gpu()

#모형 인스턴스 생성 및 트래이너, loss 정의 
model = SentClassificationModelAtt(vocab_size = len(vocab.idx_to_token), num_embed=vocab.embedding.idx_to_vec.shape[1], 
                                   hidden_size=60)


In [95]:
model.initialize(mx.init.Xavier(),ctx=ctx)
#pre-trained 된 임베딩은 모델 초기화 이후 적용한다. 
model.embed.weight.set_data(vocab.embedding.idx_to_vec.as_in_context(ctx))
#임베딩은 추가적으로 학습하지 않게 함 
#model.embed.collect_params().setattr('grad_req', 'null')
model.hybridize()

In [96]:
trainer = gluon.Trainer(model.collect_params(), 'RMSprop')
loss = gluon.loss.SoftmaxCrossEntropyLoss()

In [97]:
def evaluate_accuracy(model, data_iter, ctx=ctx):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iter):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = model(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return(acc.get()[1])

In [98]:
def calculate_loss(model, data_iter, loss_obj, ctx=ctx):
    test_loss = []
    for i, (te_data, te_label) in enumerate(data_iter):
        te_data = te_data.as_in_context(ctx)
        te_label = te_label.as_in_context(ctx)
        te_output = model(te_data)
        loss_te = loss_obj(te_output, te_label)
        curr_loss = nd.mean(loss_te).asscalar()
        test_loss.append(curr_loss)
    return(np.mean(test_loss))

In [99]:
epochs = 6


tot_test_loss = []
tot_test_accu = []
tot_train_loss = []
for e in range(epochs):
    train_loss = []
    #batch training 
    for i, (data, label) in enumerate(tqdm(train_dataloader)):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = model(data)
            loss_ = loss(output, label)
            loss_.backward()
        trainer.step(data.shape[0])

        curr_loss = nd.mean(loss_).asscalar()
        train_loss.append(curr_loss)

    #caculate test loss
    test_loss = calculate_loss(model, test_dataloader, loss_obj = loss, ctx=ctx) 
    test_accu = evaluate_accuracy(model, test_dataloader, ctx=ctx)

    print("Epoch %s. Train Loss: %s, Test Loss : %s, Test Accuracy : %s" % (e, np.mean(train_loss), test_loss, test_accu))    
    tot_test_loss.append(test_loss)
    tot_train_loss.append(np.mean(train_loss))
    tot_test_accu.append(test_accu)
    

  0%|          | 3/1800 [00:00<01:17, 23.13it/s]

Epoch 0. Train Loss: 0.39254323, Test Loss : 0.33557877, Test Accuracy : 0.8536


  0%|          | 3/1800 [00:00<01:16, 23.53it/s]

Epoch 1. Train Loss: 0.32112622, Test Loss : 0.31329557, Test Accuracy : 0.864


  0%|          | 3/1800 [00:00<01:16, 23.48it/s]

...

Epoch 3. Train Loss: 0.28251287, Test Loss : 0.300181, Test Accuracy : 0.8739


  0%|          | 3/1800 [00:00<01:16, 23.38it/s]

Epoch 4. Train Loss: 0.26984212, Test Loss : 0.29595414, Test Accuracy : 0.87375


100%|██████████| 1800/1800 [01:15<00:00, 23.88it/s]


Epoch 5. Train Loss: 0.25895712, Test Loss : 0.2962259, Test Accuracy : 0.87335
